# Project 1: MTA Trunstile Data (Exploratory Data Analysis) 
## Chicago - Winter 2020 (chi20_ds13): 
### Team #1 - Ake Paramadilok, Andrew Way, Anthony Ghabour
***

## 1. Objective

Use MTA subway (and other) data to help optimize the placement of street teams at entrances to subway stations to identify individuals passionate about technology who might attend a gala at the beginning of the summer.

Use python and pandas to perform exploratory data analysis and create visualizations via Matplotlib & Seaborn.

## 2. Workbook Setup & Data Extraction

In [1]:
# Import packages
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
from datetime import datetime as dt
import matplotlib.pyplot as plt
import folium
import geopy.geocoders
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = 'Metis MTA Project 1'
geopy.geocoders.options.default_timeout = 15
geolocator = Nominatim()

%matplotlib inline

ModuleNotFoundError: No module named 'folium'

####     MTA Data Source: http://web.mta.info/developers/turnstile.html

From http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt:

    Field Description

    C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS

    C/A      = Control Area (A002)
    UNIT     = Remote Unit for a station (R051)
    SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)
    STATION  = Represents the station name the device is located at
    LINENAME = Represents all train lines that can be boarded at this station
               Normally lines are represented by one character.  LINENAME 456NQR repersents train server 
               for 4, 5, 6, N, Q, and R trains.
    DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   
    DATE     = Represents the date (MM-DD-YY)
    TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
    DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
               1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
               2. Additionally, there may be a "RECOVR AUD" entry: This refers to missed audit that was recovered. 
    ENTRIES  = The comulative entry register value for a device
    EXIST    = The cumulative exit register value for a device


Analysis will focus on MTA activity occuring in late Spring (four weeks ending May 25, 2019) in advance of gala event in early Summer.

In [2]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

week_nums = [190504, 190511, 190518, 190525]
MTA_df = get_data(week_nums).reset_index(drop=True)

## 3. Data Wrangling & Cleanup
Let's perform some initial high-level checks. What elements are provided? 

In [ ]:
MTA_df.columns

In [ ]:
MTA_df.columns = [column.strip() for column in MTA_df.columns]
MTA_df.columns

Confirm data reflects the period of interest.

In [ ]:
MTA_df.DATE.value_counts().sort_index()

What are the applicable data types? Is there any missing information?

In [ ]:
MTA_df.info()

Note that date and time are provided as text. Let's combine and convert that information into a single column as a datetime object. Additionally, our analysis will not need to consider time to the nearest minute, so we'll drop minutes and seconds, only keeping time information in terms of hours.

In [ ]:
MTA_df["DATE_TIME"] = pd.to_datetime(MTA_df.DATE + " " + MTA_df.TIME, format="%m/%d/%Y %H:%M:%S")
MTA_df["DATE_TIME"] = MTA_df["DATE_TIME"].transform(lambda x: x.replace(minute = 0, second = 0))

Confirm transformation above worked as intended.

In [ ]:
MTA_df.info()

Next, visually inspect and review a few sample records - first, last, and/or random entries.

In [ ]:
display(MTA_df.head())
display(MTA_df.tail())
display(MTA_df.sample(5))

Turnstiles are uniquely identified by a combination of four elements. Let's create a unique identifier to facilitate manipulating later on.<br/> Similarly, sometimes distinct stations (i.e. on different lines) have the same name. 

In [ ]:
MTA_df["TURNSTILE"] = (
    MTA_df['C/A'] + ' ' +
    MTA_df['UNIT'] + ' ' +
    MTA_df['SCP'] + ' ' +
    MTA_df['STATION'])

MTA_df["STATION"] = (
    MTA_df['STATION'] + ' ' +
    MTA_df['LINENAME'] )

In [ ]:
MTA_df.DESC.value_counts()

"Recover audits" are scarce relative to "Regular" audits (less than 0.5% of all records) and may be more likely to involve irregular information, so let's ignore them. 

In [ ]:
MTA_df = MTA_df[MTA_df.DESC == "REGULAR"]

Drop columns we no longer need (much of the information has been incorporated into other columns by now). 

In [ ]:
MTA_df = MTA_df.drop(["C/A", "UNIT", "SCP", "DESC", "DIVISION", "LINENAME"], axis=1, errors="ignore")

Sanity Check to verify that "TURNSTILE", "DATE_TIME" combinations are unique.

In [ ]:
MTA_df = MTA_df.sort_values("DATE_TIME")
(MTA_df 
 .sort_values("DATE_TIME")
 .groupby(["TURNSTILE", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head()

Let's take a closer look.

In [ ]:
mask = ((MTA_df["TURNSTILE"] == "R210 R044 00-03-00 BROOKLYN BRIDGE") & 
        (MTA_df["DATE_TIME"].dt.date == datetime.datetime(2019, 5, 14).date()))

MTA_df[mask].head(10)

Note that there are multiple entries for the same "hour" with small increases in entries and exits. This is easily cleaned up as follows.

In [ ]:
MTA_df.drop_duplicates(subset = ["TURNSTILE", "DATE_TIME"], keep="last", inplace = True)

In [ ]:
mask = ((MTA_df["TURNSTILE"] == "R210 R044 00-03-00 BROOKLYN BRIDGE") & 
        (MTA_df["DATE_TIME"].dt.date == datetime.datetime(2019, 5, 14).date()))

MTA_df[mask].head(10)

Confirm ALL duplicates properly removed.

In [ ]:
(MTA_df 
 .sort_values("DATE_TIME")
 .groupby(["TURNSTILE", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

Since time data provided not especially granular, let's separate each day into morning and evening commutes. 

In [ ]:
MTA_df.loc[MTA_df.DATE_TIME.dt.time < datetime.time(13), 'COMMUTE'] = "Morning"
MTA_df.loc[MTA_df.DATE_TIME.dt.time >= datetime.time(13), 'COMMUTE'] = "Evening"

Let's see what we have.

In [ ]:
display(MTA_df.head())

Since entry and exit information is cumulative, need to shift and take differences in order to get net counts for each day, turnstile, commute.

In [ ]:
MTA_df[["PREV_DATE_TIME", "PREV_ENTRIES", "PREV_EXITS"]] = (
    MTA_df.groupby(["TURNSTILE"])["DATE_TIME", "ENTRIES", "EXITS"]
          .transform(lambda grp: grp.shift(1)))
MTA_df.describe()

In [ ]:
MTA_df.describe()

Street teams won't care if people are coming or going, so we'll combine entries and exits to determine "total targets" at each station and limit to a reasonable amount per time period (10,000).

In [ ]:
def get_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we are not giving a counter that's too big
        return 0
    return counter

MTA_df["TOTAL_TARGETS"] = MTA_df.apply(get_counts, axis=1, max_counter=1000000)
MTA_df = MTA_df[(MTA_df.TOTAL_TARGETS <= 10000)]

In [ ]:
MTA_df.describe()

Create final, clean data set for exploratory analysis.  Export as csv to handoff to teammate for additional analysis and plotting.

In [ ]:
MTA_EDA_df = MTA_df[["STATION", "DATE", "COMMUTE", "TOTAL_TARGETS"]]
MTA_EDA_df.to_csv(r'MTA_EDA.csv', index = None, header=True)

## 4. Exploratory Analysis
### Importing Data and Adding Appropriate Columns

In [ ]:
# Import csv into a dataframe object and preview contents

filename = 'MTA_EDA.csv'
mta_data = pd.read_csv(filename)
mta_data.head()

In [ ]:
# Check data characteristics
mta_data.describe()

In [ ]:
# Convert DATE column to datetime obj
mta_data['DATE'] = pd.to_datetime(mta_data['DATE'])

# Create WEEKDAY column with string describing day of week
mta_data['WEEKDAY'] = mta_data['DATE'].dt.day_name()
mta_data.head()

In [ ]:
# Create column with Day Type i.e. weekend or weekday
# mta_data now has all columns we need for analysis

weekend_days = ['Saturday','Sunday']
mta_data['DAY_TYPE'] = mta_data['WEEKDAY'].apply(lambda x: 'weekend' if x in weekend_days else 'weekday') 
mta_data.head()

In [ ]:
# Group dataframe by STATION, DATE, COMMUTE, DAY_TYPE and deep copy to mta_grp

mta_grp = (mta_data.groupby(['STATION','DATE','COMMUTE','DAY_TYPE'])
                  .sum()
                  .reset_index()
                  .sort_values("TOTAL_TARGETS", ascending=False)
                  .copy())
mta_grp

### Organize Dataframes for Weekend vs Weekday Rideship Scatter Plots

In [ ]:
# Create new grouping of dataframe that will be passed into pivot function for use in scatter plot generation

to_pivot = (mta_grp.groupby(['STATION','DAY_TYPE'])
                  .mean()
                  .reset_index()
                  .sort_values("TOTAL_TARGETS", ascending=False))
to_pivot.head(15)
display(to_pivot.describe())

In [ ]:
# Pivot Day_Type data into their own columns containing new columns of Weekday and Weekend housing 
# respective Total_Target values in preparatoin for scatter plot creation

scatter_data = to_pivot.pivot('STATION','DAY_TYPE','TOTAL_TARGETS')

In [ ]:
# Sort dataframe for descending weekday volume and save first 10 rows to create a scatter for only top 10 stations.
scatter_data_Top10 =(scatter_data.sort_values("weekday", ascending=False)
                                .round()
                                .head(10))

scatter_data_Top10

In [ ]:
# Sort dataframe for descending weekend volume figures just to see it
(scatter_data.sort_values("weekend", ascending=False)
            .round()
            .head(10))

## 5. Plotting and Visualization

In [ ]:
# Create scatter plot of average daily station traffic on weekends vs. weekdays at ALL stations

%matplotlib inline
weekend_day_scat = scatter_data.plot.scatter('weekday','weekend');

plt.xlim([0,85000]);
plt.ylim([0,85000]); 
#make y-axis scale even with x-axis scalling to emphasize tilt in data towards commuters (weekday rider)
plt.title('Average Daily Volume by Station')

figure = weekend_day_scat.get_figure()    
figure.savefig('Weekday vs Weekend Traffic', dpi=400, bbox_inches='tight',pad_inches=.25,)

In [ ]:
# Create scatter plot of top 10 stations with highest weekday traffic vs their weekend traffic
# Chart not useful and was not added to presentation

%matplotlib inline
weekend_day_scat = scatter_data_Top10.plot.scatter('weekday','weekend');
plt.xlim([0,80000]);
plt.ylim([0,80000]);
plt.title('Average Daily Volume by Station')
figure = weekend_day_scat.get_figure()    
figure.savefig('Weekday vs Weekend Traffic_top10', dpi=400, bbox_inches='tight',pad_inches=.25,)

### Organize Dataframe for Horizontal Bar Charts

In [ ]:
# Group Dataframe by Station and Commute in prep for pivot and save as deep copy after reseting indicies.

mta_grp2 = (mta_grp.groupby(['STATION','COMMUTE'])
                   .mean()
                   .sort_values("TOTAL_TARGETS", ascending=False)
                   .reset_index() 
                   .copy())
mta_grp2

In [ ]:
# Create dataframe for PM Ridership Bar Chart for Top 10 stations and pivot data to columns for use in plt

mta_grp2_pivot=mta_grp2.pivot('STATION','COMMUTE','TOTAL_TARGETS').reset_index()
mta_evening_10 = mta_grp2_pivot.sort_values("Evening", ascending=False)\
    .rename(columns={'Evening':'Average PM Ridership','Morning':'Average AM Ridership'})
mta_evening_10.round()

In [ ]:
# Sort dataframe for AM Ridership Bar Chart for Top 10 stations

mta_morning_10 = (mta_grp2_pivot.sort_values("Morning", ascending=False)
                               .reset_index()
                               .rename(columns={'Evening':'Average PM Ridership',\
                                                'Morning':'Average AM Ridership'}))
mta_morning_10.round()

In [ ]:
# TOP 20 Stations - volume per day

mta_grp3 = (mta_grp2.groupby(['STATION'])
            .sum()
            .reset_index()
            .sort_values("TOTAL_TARGETS", ascending=False)
            .rename(columns={'TOTAL_TARGETS':'Average Daily Ridership'}))
mta_top_10 = mta_grp3.head(10)
mta_top_10.to_csv(r'MTA_Top10.csv', index = None, header=True)
mta_top_10

### Create Function to Create Bar Charts

In [ ]:
# Horizontal bar chart of Top Stations

def top_stations(dataframe, num_stations_to_include, col_x_data, col_y_data, title, save_image_name):
    
    dataframe = dataframe.head(num_stations_to_include)
    top_station_all = (sns.barplot(x = col_x_data, 
                                   y = col_y_data, 
                                   data = dataframe, 
                                   orient="h")
                                   .set_title(title));
    
    # store figure to name
    figure = top_station_all.get_figure()    
    
    # save figure and export, bbox_inches "tight" to define margins, which were initially cutting off station names
    figure.savefig(save_image_name, dpi=400, bbox_inches='tight',pad_inches=.25)

In [ ]:
# Reset index and rename column in order to control bar chart axis name ('Average Daily Ridership')

(mta_grp3.reset_index()
         .round()
         .rename(columns={'TOTAL_TARGETS':'Average Daily Ridership'})
         .head(10));

### Create Bar Charts For AM and PM Ridership Using top_stations( ) Function

In [ ]:
top_stations(dataframe = mta_grp3, 
             num_stations_to_include = 10,
             col_x_data = 'Average Daily Ridership', 
             col_y_data = 'STATION',
             title = 'Ave Daily Ridership per Station', 
             save_image_name = 'Ave_Daily_Vol');

In [ ]:
# Create PM Ridership Bar Chart
top_stations(dataframe = mta_evening_10, 
             num_stations_to_include = 10,
             col_x_data = 'Average PM Ridership', 
             col_y_data = 'STATION',
             title = 'Ave Evening Ridership per Station', 
             save_image_name = 'Top_PM_Vol');

In [ ]:
# Create AM Ridership Bar Chart
top_stations(dataframe = mta_morning_10, 
             num_stations_to_include = 10,
             col_x_data = 'Average AM Ridership', 
             col_y_data = 'STATION',
             title = 'Ave Morning Ridership per Station', 
             save_image_name = 'Top_AM_Vol');

## 6. Maps

In [ ]:
offices_sqft = pd.read_csv('Location Data - Largest SQFT.csv')
offices_pop = pd.read_csv('Location Data - Most Employees.csv')
station_data = pd.read_csv('NYC_Transit_Subway_Entrance_And_Exit_Data.csv')

In [ ]:
def add_coords(dataframe):
    '''
    Accepts dataframe of location data with address column,
    Adds 2 new float columns to dataframe with latitude and longitude coordinates
    for later use in folium map
    
    Relies on geopy geocode information
    Geopy returns 2 cols:
        0: str containing descriptive data related to address
        1: tuple containing latitude and longitude as floats
    
    '''
    lat,lon = [],[]
    for i in range(len(dataframe)):
        location = geolocator.geocode(dataframe['Location'][i])
        lat.append(location[-1][0])
        lon.append(location[-1][1])

    dataframe['Latitude'] = lat
    dataframe['Longitude'] = lon

In [ ]:
add_coords(offices_sqft)
add_coords(offices_pop)
#No need to add coordinates to the MTA data, it's already in there

In [ ]:
station_data.rename(columns=lambda x: x.strip().replace(' ','_'), inplace=True)

In [ ]:
station_data.columns

In [ ]:
for i in range(1,12):
    station_data[f'Route{i}']=station_data[f'Route{i}'].astype(str)
    station_data[f'Route{i}']=station_data[f'Route{i}'].str.replace('nan','')

    
    '''
    Some MTA lines have numbers for names
    and they're in this df as either floats or ints.
    This code converts them to strings to concatenate into
    an identifying code to prevent overlaps in station names
    '''


In [ ]:
station_data['Station_ID'] =\
station_data['Division']+' '+\
station_data['Line']+' '+\
station_data['Station_Name']+' '+\
station_data['Route1']+\
station_data['Route2']+\
station_data['Route3']+\
station_data['Route4']+\
station_data['Route5']+\
station_data['Route6']+\
station_data['Route7']+\
station_data['Route8']+\
station_data['Route9']+\
station_data['Route10']+\
station_data['Route11']

#concatenating individual info into identifying column

In [ ]:
station_legend = station_data[['Station_Name','Station_ID','Station_Latitude','Station_Longitude']]
station_legend.drop_duplicates(inplace=True)
station_legend.reset_index(drop=True,inplace=True)

#This block creates a legend with just the columns I want,
#then removes the repeat entries and resets the index

In [ ]:
mta_top_10.reset_index(drop=True,inplace=True)

In [ ]:
mta_top_10

This next section is pretty ugly and manual.  Given that these datasets have markedly different conventions for name formatting, I'll be poking around to find the correct coordinates and then attach them to the appropriate entry, based on my own domain knowledge of the MTA.  I chose this tactic because the connection between the turnstile 'STATION' entry and station legend 'Station_Name' entry was rather ambiguous. I also knew this  would only be for 10 items, and with the understanding that it isn't scalable much further beyond this quantity.



_A Way_

In [ ]:
mta_top_10.iloc[0]

In [ ]:
def legend_finder(search_query):
    return station_legend[station_legend['Station_Name'].str.contains(search_query)]

    '''
    This function is a shortcut to search for a string in a station name
    Input: string, one you expect to be in the station name
    Output: df containing entries where input is present in station name
    '''

In [ ]:
legend_finder('Grand Central')

8 of our 10 top stations are junctions, stations that serve 2 or more unique lines.  This means that our station legend (and the datasets preceding it) have seperate entries and coordinates for each section of platforms.  For example, in the case of Grand Central, there are 3 entries:

Station Name | Description
:--|:--
IRT Lexington Grand Central-42nd St | the 4/5/6 trains running North/South
IRT Flushing Grand Central-42nd St GS4567 | the 7 train running East/West
IRT 42nd St Shuttle Grand Central | the S shuttle, a half-length train connecting Grand Central to Times Square

Given that our geographical coordinates extend to 6 decimal places, and that differences in geographical coordinates at the 4th decimal place are a maximum of ~30 feet at the equator, these differences are negligible.

I'll be using my best judgement to focus on the platform that is closest to ground level and closest to the center of the station, based on renderings from http://www.projectsubwaynyc.com/.

_A Way_

In [ ]:
lat_list = []
lon_list = [] #we will add coordinates in order to these two lists, then add them to our top 10

In [ ]:
def coord_add(station_string):
    lat_list.append(station_legend[station_legend.Station_ID == station_string]
                              ['Station_Latitude'])
    lon_list.append(station_legend[station_legend.Station_ID == station_string]
                              ['Station_Longitude'])
    return 'coords_added'

In [ ]:
coord_add('IRT Lexington Grand Central-42nd St GS4567') #4/5/6 platform

In [ ]:
mta_top_10.iloc[1]

In [ ]:
legend_finder('34th')

In [ ]:
coord_add('BMT Broadway 34th St BDFMNQR') #N/Q/R/W platform

In [ ]:
mta_top_10.iloc[2]

In [ ]:
legend_finder('42nd')

In [ ]:
coord_add('IND 8 Avenue 42nd St ACENQRS1.02.03.07.0') #'PORT-AUTH' implies 8th avenue line

In [ ]:
mta_top_10.iloc[3]

In [ ]:
legend_finder('34th')

In [ ]:
coord_add('IND 8 Avenue 34th St ACE') #Not a junction, no comparisons needed

In [ ]:
mta_top_10.iloc[4]

In [ ]:
legend_finder('Union')

In [ ]:
coord_add('BMT Canarsie Union Square LNQR456') #L train straddles B'way and Lexington lines

In [ ]:
mta_top_10.iloc[5]

In [ ]:
legend_finder('Times')

In [ ]:
coord_add('BMT Broadway Times Square-42nd St ACENQRS1.02.03.07.0')
# These lines are so close that this one doesn't really matter

In [ ]:
mta_top_10.iloc[6]

In [ ]:
legend_finder('59th')

In [ ]:
coord_add('IND 8 Avenue 59th St ABCD1') # 4 of 5 lines at this station are at the aforementioned platform

In [ ]:
mta_top_10.iloc[7]

In [ ]:
legend_finder('Fulton')

In [ ]:
coord_add('BMT Nassau Fulton St ACJZ2345.0') #J/Z Nassau line is most central at this location

In [ ]:
mta_top_10.iloc[8]

In [ ]:
legend_finder('47-50')

In [ ]:
coord_add('IND 6 Avenue 47-50th Sts Rockefeller Center BDFM') #another non-junction

In [ ]:
mta_top_10.iloc[9]

In [ ]:
legend_finder('Flushing')

In [ ]:
coord_add('IRT Flushing Flushing-Main St 7') #non-junction

In [ ]:
mta_top_10['Latitude']=lat_list
mta_top_10['Longitude']=lon_list

In [ ]:
mta_top_10.head()

In [ ]:
nycmap = folium.Map(location=[40.73,-73.982155], 
                    width=1024, 
                    height=760, 
                    tiles="cartodbpositron", 
                    zoom_start=12)

for i in range(len(mta_top_10)):
    folium.Circle([mta_top_10['Latitude'][i], mta_top_10['Longitude'][i]],
                  popup=mta_top_10['STATION'][i],
                  radius = 300,
                  color = 'crimson',
                  fill = True
                  ).add_to(nycmap)


for i in range(len(offices_pop)):
    folium.Circle([offices_pop.iloc[i]['Latitude'], offices_pop.iloc[i]['Longitude']],
                  popup=offices_pop.iloc[i]['Company'],
                  radius= 10,
                  fill=True,
                  color = 'blue'
                  ).add_to(nycmap)

for i in range(len(offices_sqft)):
    folium.Circle([offices_sqft.iloc[i]['Latitude'], offices_sqft.iloc[i]['Longitude']],
                   popup=offices_sqft.iloc[i]['Company'],
                   radius= 10,
                   fill=True,
                   color = 'blue').add_to(nycmap)

nycmap.save('nycmap.html')
nycmap